In [39]:
# HIDDEN
from datascience import *
from datascience.predicates import are
import numpy as np
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

def load_and_clean_table(url):
    """Loads a table about Marvel or DC comics from fivethirtyeight's
    GitHub repository, and cleans up a few formatting details."""
    tbl = Table.read_table(url)
    if "Year" in tbl.labels:
        tbl.relabel("Year", "YEAR")
        tbl.update({'PUBLISHER': 'Marvel'})
    else:
        tbl.update({'PUBLISHER': 'DC'})
    tbl.relabel("name", "NAME")
    tbl.update({"APPEARANCES": np.nan_to_num(tbl.column("APPEARANCES"))})
    tbl.update({"GENDER": np.char.replace(tbl.column("SEX"), "nan", "Unknown")})
    tbl.update({"GENDER": np.char.replace(tbl.column("GENDER"), " Characters", "")})
    
    def extract_month(date_text):
        import dateutil.parser
        try:
            return dateutil.parser.parse(date_text).month
        except:
            return "Unknown"
    
    tbl.update({"MONTH": tbl.apply(extract_month, "FIRST APPEARANCE")})
    tbl.update({"MONTH": tbl.apply(lambda d: int(d) if d != "Unknown" else -1, "MONTH")})
    tbl = tbl.select("PUBLISHER", "NAME", "GENDER", "APPEARANCES", "YEAR", "MONTH")
    for l in tbl.labels:
        tbl.relabel(l, l.capitalize())
    tbl = tbl.where(~np.isnan(tbl.column("Year")))
    return tbl

In this example, we will explore statistics about characters in comic books.  A 2014 [article](https://fivethirtyeight.com/features/women-in-comic-books/) on FiveThirtyEight examined the representation of women in comics published by DC and Marvel, the two largest publishers of comic books.  The authors collected a dataset about characters from the publishers' [Wikia](http://marvel.wikia.com/Main_Page) [pages](http://dc.wikia.com/wiki/Main_Page) and made it publicly available on [GitHub](https://github.com/fivethirtyeight/data/tree/master/comic-characters).  Using Python, we can load the data directly from the web.

This example is meant to illustrate some of the broad themes of this text. You will see code examples like this throughout this text. For now, don't worry if the details of the program don't yet make sense. In fact, important details of this program have been hidden from view. Instead, focus on interpreting the results displayed below the code. Later sections of the text will describe most of the features of the Python programming language used below.

First, we read the data about each publisher. The `#` symbol below starts a comment, which is ignored by the computer but helpful for people reading the code. The `=` symbols assign a name on the left to the result of some computation described on the right. A *uniform resource locator* or *URL* is an address on the Internet for some content; in this case, a table of information about comic book characters. (In Python, a name cannot contain any spaces, and so we will often use an underscore `_` to stand in for a space.) So we name the URL for the Marvel data `marvel_url`, and we name the loaded dataset itself `marvel`.

In [ ]:
# Load datasets from FiveThirtyEight about comics:

marvel_url = "https://github.com/fivethirtyeight/data/raw/master/comic-characters/marvel-wikia-data.csv"
dc_url = "https://github.com/fivethirtyeight/data/raw/master/comic-characters/dc-wikia-data.csv"

marvel = load_and_clean_table(marvel_url)
dc = load_and_clean_table(dc_url)

To display one of the tables, we write its name:

In [37]:
marvel

Publisher,Name,Gender,Appearances,Year,Month
Marvel,Spider-Man (Peter Parker),Male,4043,1962,8
Marvel,Captain America (Steven Rogers),Male,3360,1941,3
Marvel,"Wolverine (James \""Logan\"" Howlett)",Male,3061,1974,10
Marvel,"Iron Man (Anthony \""Tony\"" Stark)",Male,2961,1963,3
Marvel,Thor (Thor Odinson),Male,2258,1950,11
Marvel,Benjamin Grimm (Earth-616),Male,2255,1961,11
Marvel,Reed Richards (Earth-616),Male,2072,1961,11
Marvel,Hulk (Robert Bruce Banner),Male,2017,1962,5
Marvel,Scott Summers (Earth-616),Male,1955,1963,9
Marvel,Jonathan Storm (Earth-616),Male,1934,1961,11


In [38]:
dc

Publisher,Name,Gender,Appearances,Year,Month
DC,Batman (Bruce Wayne),Male,3093,1939,5
DC,Superman (Clark Kent),Male,2496,1986,10
DC,Green Lantern (Hal Jordan),Male,1565,1959,10
DC,James Gordon (New Earth),Male,1316,1987,2
DC,Richard Grayson (New Earth),Male,1237,1940,4
DC,Wonder Woman (Diana Prince),Female,1231,1941,12
DC,Aquaman (Arthur Curry),Male,1121,1941,11
DC,Timothy Drake (New Earth),Male,1095,1989,8
DC,Dinah Laurel Lance (New Earth),Female,1075,1969,11
DC,Flash (Barry Allen),Male,1028,1956,10


Each table has a record (displayed in one horizontal row) for every character in the dataset from its respective publisher.  Each vertical column contains a piece of information about each character.  We can see, for example, that Batman has appeared 3,093 times in DC comics since his appearance in May (month 5) 1939.